<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/15.%20Chapter15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 15: Processing Sequences Using RNN and CNN

Bab ini memperkenalkan **Recurrent Neural Networks (RNNs)**, sebuah kelas jaringan saraf yang dirancang khusus untuk menangani data sekuensial atau urutan, seperti data deret waktu (*time series*), teks, atau audio. Berbeda dengan jaringan saraf sebelumnya, RNN memiliki koneksi yang mengarah ke belakang, memberinya semacam "memori".

* **Recurrent Neurons dan Layers**:
    * Neuron rekuren menerima input dari langkah waktu saat ini dan outputnya sendiri dari langkah waktu sebelumnya. Ini memungkinkan jaringan untuk mengingat informasi dari masa lalu.
    * Ketika RNN di-"buka" sepanjang waktu (*unrolled through time*), ia terlihat seperti jaringan saraf dalam biasa, di mana bobotnya digunakan bersama di setiap langkah waktu.
    * **Memory Cell**: Bagian dari jaringan yang menyimpan *state* antar langkah waktu. Sel RNN sederhana hanya mampu mengingat pola jangka pendek.

* **Arsitektur Input dan Output**: RNN sangat fleksibel:
    * **Sequence-to-Sequence**: Menerima urutan dan menghasilkan urutan (misalnya, peramalan cuaca).
    * **Sequence-to-Vector**: Menerima urutan dan menghasilkan satu vektor (misalnya, analisis sentimen).
    * **Vector-to-Sequence**: Menerima satu vektor dan menghasilkan urutan (misalnya, membuat *caption* untuk gambar).
    * **Encoder-Decoder**: Gabungan dari *sequence-to-vector* dan *vector-to-sequence* (misalnya, penerjemahan mesin).

* **Tantangan dalam Melatih RNN**:
    * **Gradien yang Tidak Stabil**: Sama seperti DNN biasa, RNN yang dalam (panjang) dapat menderita masalah *vanishing* atau *exploding gradient*.
    * **Memori Jangka Pendek yang Terbatas**: Sel RNN sederhana cenderung "melupakan" input yang sudah sangat lama.

* **Mengatasi Masalah Memori Jangka Pendek**:
    * **LSTM (Long Short-Term Memory) Cells**: Sel yang lebih kompleks dengan *state* jangka panjang dan jangka pendek. Ia memiliki tiga "gerbang" (*gates*): *forget gate*, *input gate*, dan *output gate*, yang secara cerdas mengatur informasi apa yang harus disimpan, dibuang, dan dibaca.
    * **GRU (Gated Recurrent Unit) Cells**: Varian yang lebih sederhana dari LSTM yang seringkali memberikan performa yang sama baiknya. Ia menggabungkan *state* jangka panjang dan pendek menjadi satu dan hanya menggunakan dua gerbang.

* **Menggunakan CNN untuk Urutan**:
    * **1D Convolutional Layers**: Lapisan konvolusional 1D dapat diterapkan pada urutan. Mereka sangat efisien dan dapat belajar mendeteksi pola lokal jangka pendek. Mereka bisa digunakan sebagai lapisan *preprocessing* untuk RNN (misalnya, untuk *downsampling*) atau digunakan sendiri.
    * **WaveNet**: Arsitektur yang hanya menggunakan lapisan konvolusional 1D yang ditumpuk, dengan *dilation rate* yang meningkat secara eksponensial. Ini memungkinkan jaringan untuk memiliki *receptive field* yang sangat besar dan belajar pola jangka panjang dengan sangat efisien.

### 1. Meramalkan Deret Waktu dengan RNN Sederhana
Kita akan membuat data deret waktu sintetis dan mencoba meramalkan nilai berikutnya.

```python
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# Fungsi untuk membuat dataset deret waktu
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

# Membuat data training, validasi, dan testing
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

# Membangun model RNN sederhana
# Keras secara default hanya mengembalikan output dari langkah waktu terakhir
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, input_shape=[None, 1])
])

# Menggunakan Dense layer sebagai output lebih fleksibel
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20), # Hanya mengembalikan output terakhir
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
# history = model.fit(X_train, y_train, epochs=20,
#                     validation_data=(X_valid, y_valid))
print("Model RNN untuk peramalan satu langkah telah dibuat.")
```

### 2. Meramalkan Beberapa Langkah ke Depan (Sequence-to-Sequence)
Kita bisa melatih RNN untuk memprediksi 10 nilai berikutnya di setiap langkah waktu. Ini menstabilkan dan mempercepat training.

```python
# Mempersiapkan target: setiap target adalah urutan 10D
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, n_steps:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, n_steps:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, n_steps:, 0]

# Membangun model Seq-to-Seq
# return_sequences=True di semua lapisan RNN
# TimeDistributed menerapkan Dense layer ke setiap langkah waktu
model_seq = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# Fungsi loss kustom untuk hanya mengevaluasi langkah waktu terakhir
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model_seq.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  metrics=[last_time_step_mse])

# history = model_seq.fit(X_train, Y_train, epochs=20,
#                         validation_data=(X_valid, Y_valid))
print("\nModel Seq-to-Seq telah dibuat.")
```

### 3. LSTM dan GRU
Untuk menangani ketergantungan jangka panjang, kita bisa mengganti `SimpleRNN` dengan `LSTM` atau `GRU`. Penggunaannya di Keras sangat mudah.

```python
# Model dengan GRU (penggunaannya sama dengan LSTM)
model_gru = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# Kompilasi dan training sama seperti sebelumnya
# model_gru.compile(...)
# model_gru.fit(...)
print("\nModel dengan lapisan GRU telah dibuat.")
```

### 4. WaveNet Menggunakan Lapisan Konvolusional 1D
Contoh arsitektur yang terinspirasi dari WaveNet menggunakan tumpukan lapisan `Conv1D` dengan `dilation_rate` yang meningkat.

```python
# Mempersiapkan data Y yang sama panjangnya dengan X
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train, Y_valid = Y[:7000], Y[7000:9000]

# Model WaveNet
model_wavenet = keras.models.Sequential()
model_wavenet.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2: # Tumpukan blok dilatasi
    model_wavenet.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                 activation="relu", dilation_rate=rate))
model_wavenet.add(keras.layers.Conv1D(filters=10, kernel_size=1)) # Lapisan output

model_wavenet.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

# history = model_wavenet.fit(X_train, Y_train, epochs=20,
#                             validation_data=(X_valid, Y_valid))
print("\nModel WaveNet telah dibuat.")
```
Lapisan konvolusional 1D dengan *causal padding* dan *dilation rate* yang meningkat memungkinkan model untuk melihat pola jangka panjang secara efisien tanpa memerlukan komputasi sekuensial dari RNN.
